In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANTITATIVE BETTING PORTFOLIO MANAGER v1.0      ")
print("=======================================================")

# ==============================================================================
# 1. USER CONFIGURATION (EDIT THIS PART)
# ==============================================================================
BANKROLL = 25.00        # Total betting budget ($)
RISK_MODE = "Sensible"    # Options: "Aggressive" (0.5), "Sensible" (0.25), "Conservative" (0.1)
MIN_EDGE = 0.05           # Minimum 5% ROI required to place a bet
MAX_STAKE_PCT = 0.06      # Max 6% of bankroll on a single match (Risk management)
MIN_PROBABILITY = 0.20    # Ignore outcomes with <20% chance (Reduces variance)

# --- MANUAL FIXTURE LIST (Team Names must match your CSV data) ---
fixtures = [
    # Format: {"Home": "Team A", "Away": "Team B", "Odds_1": HomeOdds, "Odds_X": DrawOdds, "Odds_2": AwayOdds}
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. SYSTEM BACKEND (DO NOT TOUCH)
# ==============================================================================

# Risk Multiplier Logic
kelly_fractions = {"Aggressive": 0.5, "Sensible": 0.25, "Conservative": 0.1}
KELLY_F = kelly_fractions.get(RISK_MODE, 0.25)

try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded. Analyzing {len(fixtures)} upcoming matches...")
except FileNotFoundError:
    print("❌ Critical Error: 'football_model_final.pkl' missing.")
    exit()

def get_stats(team):
    # Find latest data point for the team
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    stats = {}
    
    # Extract rolling features
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points']
    for f in roll_feats:
        col = f"{prefix}roll_{f}"
        if col in last: stats[f] = last[col]
        else: stats[f] = 0
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    
    # 1. Fetch Data
    h_stats = get_stats(h_team)
    a_stats = get_stats(a_team)
    h_elo = current_elos.get(h_team, 1500)
    a_elo = current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats:
        print(f"⚠️  Missing data for {h_team} vs {a_team}. Skipping.")
        continue

    # 2. Build Feature Vector (Exact match to training)
    input_data = {
        'diff_elo': (h_elo + 65) - a_elo,
        'home_elo': h_elo,
        'away_elo': a_elo,
        'diff_rest': 0, # Neutral assumption
        'home_roll_team_xg': h_stats['team_xg'],
        'away_roll_team_xg': a_stats['team_xg']
    }
    
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points']
    for feat in roll_feats:
        input_data[f"diff_{feat}"] = h_stats[feat] - a_stats[feat]

    # Predict
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    
    # Probabilities: [Away, Draw, Home] (Standard sklearn order for 0,1,2 classes)
    # Check your model encoding! Assuming: 0=Away, 1=Draw, 2=Home based on previous code
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]
    
    # 3. Find Value (Compare vs Odds)
    bets_for_match = []
    
    # Option 1: Home
    edge_home = (p_home * f['Odds_1']) - 1
    if edge_home > MIN_EDGE and p_home > MIN_PROBABILITY:
        bets_for_match.append({'Type': 'HOME', 'Team': h_team, 'Odds': f['Odds_1'], 'Prob': p_home, 'Edge': edge_home})
        
    # Option 2: Draw
    edge_draw = (p_draw * f['Odds_X']) - 1
    if edge_draw > MIN_EDGE and p_draw > MIN_PROBABILITY:
        bets_for_match.append({'Type': 'DRAW', 'Team': 'Draw', 'Odds': f['Odds_X'], 'Prob': p_draw, 'Edge': edge_draw})
        
    # Option 3: Away
    edge_away = (p_away * f['Odds_2']) - 1
    if edge_away > MIN_EDGE and p_away > MIN_PROBABILITY:
        bets_for_match.append({'Type': 'AWAY', 'Team': a_team, 'Odds': f['Odds_2'], 'Prob': p_away, 'Edge': edge_away})
    
    # 4. Pick Best Bet for this match (Avoid conflicting bets)
    if bets_for_match:
        # Sort by Edge and pick the best one
        best_bet = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        best_bet['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best_bet)

# ==============================================================================
# 4. PORTFOLIO ALLOCATION (KELLY CRITERION)
# ==============================================================================
print("\n⚙️ Calculating Optimal Stakes (Kelly Criterion)...")

portfolio = pd.DataFrame(opportunities)

if not portfolio.empty:
    def calculate_stake(row):
        # Kelly Formula: f = (bp - q) / b
        # b = odds - 1
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        
        f = ((b * p) - q) / b
        
        # Apply Safety Multipliers
        f_safe = f * KELLY_F
        
        # Apply Max Cap
        return max(0, min(f_safe, MAX_STAKE_PCT))

    portfolio['Kelly_Pct'] = portfolio.apply(calculate_stake, axis=1)
    portfolio['Stake'] = portfolio['Kelly_Pct'] * BANKROLL
    portfolio['Est_Return'] = portfolio['Stake'] * (portfolio['Odds'] - 1)
    
    # Filter out zero stakes (if any negative edges slipped through)
    portfolio = portfolio[portfolio['Stake'] > 0.5].sort_values(by='Stake', ascending=False)
    
    # Output
    print(f"\n📋 BETTING SLIP ({RISK_MODE} Mode - Bank: ${BANKROLL})")
    print("-" * 100)
    print(f"{'MATCH':<35} | {'SELECTION':<15} | {'ODDS':<5} | {'PROB':<5} | {'EDGE':<5} | {'STAKE':<8} | {'POT. PROFIT'}")
    print("-" * 100)
    
    total_invested = 0
    total_ev = 0
    
    # 2. Rename columns for clean display
    display_df = portfolio.copy()
    display_df = display_df[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Est_Return']]
    display_df.columns = ['Match', 'Bet Type', 'Team Selection', 'Odds', 'Win Prob', 'Edge (ROI)', 'Stake ($)', 'Pot. Profit ($)']
    
    # 3. Apply Professional Quant Styling
    styled_table = (display_df.style
        .format({
            'Odds': '{:.2f}',
            'Win Prob': '{:.1%}',
            'Edge (ROI)': '{:.1%}',
            'Stake ($)': '${:.2f}',
            'Pot. Profit ($)': '${:.2f}'
        })
        # Green background for higher Probability (Confidence)
        .background_gradient(subset=['Win Prob'], cmap='Greens', vmin=0.2, vmax=0.8)
        
        # Blue background for Edge (Value)
        .background_gradient(subset=['Edge (ROI)'], cmap='Blues', vmin=0.05, vmax=0.5)
        
        # Data Bars for Stake (Visualizing Money Allocation)
        .bar(subset=['Stake ($)'], color='#d65f5f', vmin=0)
        
        # Bold the Team Name
        .set_properties(subset=['Team Selection'], **{'font-weight': 'bold'})
        
        .set_caption("🚀 OPTIMIZED QUANT PORTFOLIO")
        .set_table_styles([
            {'selector': 'th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('background-color', '#f4f4f4')]},
            {'selector': 'td', 'props': [('text-align', 'center'), ('padding', '8px')]},
            {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold'), ('color', '#333')]}
        ])
    )
    
    # 4. Display
    from IPython.display import display
    display(styled_table)

        
    print("-" * 100)
    print(f"💵 REMAINING CASH:   ${BANKROLL - total_invested:.2f}")
    print(f"💰 TOTAL INVESTMENT: ${total_invested:.2f}  ({(total_invested/BANKROLL)*100:.1f}% of Bankroll)")
    print(f"📈 EXPECTED VALUE:   ${total_ev:.2f} (Theoretical Profit)")
    print(f"💵 EXPECTED CASH (after profit):   ${BANKROLL +total_ev:.2f}")

else:
    print("\n📉 No Value Bets Found.")
    print("The model thinks the bookies have priced all these games correctly (or you need lower thresholds).")

   🤖 QUANTITATIVE BETTING PORTFOLIO MANAGER v1.0      
✅ Model Loaded. Analyzing 10 upcoming matches...

⚙️ Calculating Optimal Stakes (Kelly Criterion)...

📋 BETTING SLIP (Sensible Mode - Bank: $25.0)
----------------------------------------------------------------------------------------------------
MATCH                               | SELECTION       | ODDS  | PROB  | EDGE  | STAKE    | POT. PROFIT
----------------------------------------------------------------------------------------------------
Chelsea vs Aston Villa              | AWAY (Aston Villa) | 4.03  | 44%  | 76.6% | $1.50    | $4.54
Crystal Palace vs Tottenham Hotspur | HOME (Crystal Palace) | 2.29  | 58%  | 33.9% | $1.50    | $1.94
Sunderland vs Leeds United          | HOME (Sunderland) | 2.58  | 51%  | 32.3% | $1.28    | $2.02
Manchester United vs Newcastle United | HOME (Manchester United) | 2.53  | 50%  | 26.3% | $1.07    | $1.64
Burnley vs Everton                  | HOME (Burnley)  | 4.01  | 35%  | 39.4% | $0.82   